In [19]:
# Load Amazon S3 data into Spark dataframe


import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 3,622 B in 1s (2,610 B/s)
Reading package lists... Done


In [20]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-04-09 17:20:15--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2023-04-09 17:20:15 (11.3 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [21]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Final-Project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [22]:
from pyspark import SparkFiles
url = "https://highered-bucket.s3.amazonaws.com/highered_merged_data.csv"
spark.sparkContext.addFile(url)
highered_data_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), header=True, inferSchema=True)
highered_data_df.show()

+----+------+-------+-----+-------+-------+-------+--------------------+------+------+-----+-------+-------+--------+----------+---------+
|Year|UNITID|FACSTAT|ARANK|HRTOTLT|HRTOTLM|HRTOTLW|              INSTNM|  CITY|STABBR|  ZIP|CONTROL|HLOFFER|INSTSIZE|  LONGITUD| LATITUDE|
+----+------+-------+-----+-------+-------+-------+--------------------+------+------+-----+-------+-------+--------+----------+---------+
|2019|100654|      0|    0|    242|    131|    111|Alabama A & M Uni...|Normal|    AL|35762|      1|      9|       3|-86.568502|34.783368|
|2019|100654|     10|    0|    242|    131|    111|Alabama A & M Uni...|Normal|    AL|35762|      1|      9|       3|-86.568502|34.783368|
|2019|100654|     10|    1|     49|     37|     12|Alabama A & M Uni...|Normal|    AL|35762|      1|      9|       3|-86.568502|34.783368|
|2019|100654|     10|    2|     50|     34|     16|Alabama A & M Uni...|Normal|    AL|35762|      1|      9|       3|-86.568502|34.783368|
|2019|100654|     10|    3|

In [44]:
# Import PySpark Pandas
import pyspark.pandas as ps

In [48]:
# Read table as a pandas dataframe
pandas_df = ps.DataFrame(highered_data_df)
pandas_df.head(10)

,Year,UNITID,FACSTAT,ARANK,HRTOTLT,HRTOTLM,HRTOTLW,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
0,2019,100654,0,0,242,131,111,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
1,2019,100654,10,0,242,131,111,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
2,2019,100654,10,1,49,37,12,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
3,2019,100654,10,2,50,34,16,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
4,2019,100654,10,3,98,46,52,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
5,2019,100654,10,4,38,11,27,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
6,2019,100654,10,5,7,3,4,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
7,2019,100654,20,0,99,68,31,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
8,2019,100654,20,1,49,37,12,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
9,2019,100654,20,2,32,22,10,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368


In [62]:
# Drop FACSTAT column
test_df = pandas_df.drop(["FACSTAT", "ARANK"], axis=1)
test_df.head(10)

,Year,UNITID,HRTOTLT,HRTOTLM,HRTOTLW,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
0,2019,100654,242,131,111,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
1,2019,100654,242,131,111,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
2,2019,100654,49,37,12,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
3,2019,100654,50,34,16,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
4,2019,100654,98,46,52,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
5,2019,100654,38,11,27,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
6,2019,100654,7,3,4,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
7,2019,100654,99,68,31,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
8,2019,100654,49,37,12,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
9,2019,100654,32,22,10,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368


In [65]:
# Drop Duplicates
test_df.drop_duplicates()
test_df.head(20)

,Year,UNITID,HRTOTLT,HRTOTLM,HRTOTLW,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
6,2019,100654,7,3,4,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
12,2019,100654,9,5,4,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
15,2019,100654,9,7,2,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
10,2019,100654,18,9,9,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
13,2019,100654,30,11,19,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
9,2019,100654,32,22,10,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
5,2019,100654,38,11,27,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
11,2019,100654,39,16,23,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
2,2019,100654,49,37,12,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
16,2019,100654,50,26,24,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368


In [80]:
test1_df = test_df[['Year', 'UNITID', 'HRTOTLT', 'HRTOTLM', 'HRTOTLW','INSTSIZE']]
test1_df.head()

,Year,UNITID,HRTOTLT,HRTOTLM,HRTOTLW,INSTSIZE
6,2019,100654,7,3,4,3
12,2019,100654,9,5,4,3
15,2019,100654,9,7,2,3
10,2019,100654,18,9,9,3
13,2019,100654,30,11,19,3


In [82]:
test1_df.groupby(['UNITID','Year'])['HRTOTLT'].sum().head(10)

UNITID  Year
141264  2019    2054
155025  2019     882
157304  2019     398
188207  2019       4
197285  2019      44
199263  2019      27
210146  2019     485
213376  2019     136
218441  2019     314
219347  2019     586
Name: HRTOTLT, dtype: int64

In [42]:
highered_data_df.dropDuplicates().show()

+----+------+-------+-----+-------+-------+-------+--------------------+------------+------+----------+-------+-------+--------+-----------+---------+
|Year|UNITID|FACSTAT|ARANK|HRTOTLT|HRTOTLM|HRTOTLW|              INSTNM|        CITY|STABBR|       ZIP|CONTROL|HLOFFER|INSTSIZE|   LONGITUD| LATITUDE|
+----+------+-------+-----+-------+-------+-------+--------------------+------------+------+----------+-------+-------+--------+-----------+---------+
|2019|100654|     10|    2|     50|     34|     16|Alabama A & M Uni...|      Normal|    AL|     35762|      1|      9|       3| -86.568502|34.783368|
|2019|100690|     43|    2|      1|      1|      0|  Amridge University|  Montgomery|    AL|36117-3553|      2|      9|       1|  -86.17401|32.362609|
|2019|100858|     20|    0|    718|    489|    229|   Auburn University|      Auburn|    AL|     36849|      1|      9|       5| -85.488258|32.599378|
|2019|101462|     42|    6|      1|      0|      1|J. F. Drake State...|  Huntsville|    AL|  

In [25]:
# Drop columns ARANK and FACSTAT
df_test = highered_data_df.drop("ARANK", "FACSTAT")
df_test.show()

+----+------+-------+-------+-------+--------------------+------+------+-----+-------+-------+--------+----------+---------+
|Year|UNITID|HRTOTLT|HRTOTLM|HRTOTLW|              INSTNM|  CITY|STABBR|  ZIP|CONTROL|HLOFFER|INSTSIZE|  LONGITUD| LATITUDE|
+----+------+-------+-------+-------+--------------------+------+------+-----+-------+-------+--------+----------+---------+
|2019|100654|    242|    131|    111|Alabama A & M Uni...|Normal|    AL|35762|      1|      9|       3|-86.568502|34.783368|
|2019|100654|    242|    131|    111|Alabama A & M Uni...|Normal|    AL|35762|      1|      9|       3|-86.568502|34.783368|
|2019|100654|     49|     37|     12|Alabama A & M Uni...|Normal|    AL|35762|      1|      9|       3|-86.568502|34.783368|
|2019|100654|     50|     34|     16|Alabama A & M Uni...|Normal|    AL|35762|      1|      9|       3|-86.568502|34.783368|
|2019|100654|     98|     46|     52|Alabama A & M Uni...|Normal|    AL|35762|      1|      9|       3|-86.568502|34.783368|


In [41]:
df_test.dropDuplicates().show()

+----+------+-------+-------+-------+--------------------+-------------+------+----------+-------+-------+--------+-----------+---------+
|Year|UNITID|HRTOTLT|HRTOTLM|HRTOTLW|              INSTNM|         CITY|STABBR|       ZIP|CONTROL|HLOFFER|INSTSIZE|   LONGITUD| LATITUDE|
+----+------+-------+-------+-------+--------------------+-------------+------+----------+-------+-------+--------+-----------+---------+
|2019|101587|     25|     11|     14|University of Wes...|   Livingston|    AL|     35470|      1|      9|       3| -88.186329|32.588132|
|2019|101879|      3|      2|      1|University of Nor...|     Florence|    AL|35632-0001|      1|      8|       3| -87.681361|34.806738|
|2019|101879|     61|     30|     31|University of Nor...|     Florence|    AL|35632-0001|      1|      8|       3| -87.681361|34.806738|
|2019|102049|    180|    114|     66|  Samford University|   Birmingham|    AL|35229-2240|      2|      9|       3| -86.791799|33.464128|
|2019|102234|      7|      2|     

In [40]:
# Create the institution size table
institution_size_df = df_test.groupBy("UNITID", "Year") \
.sum("HRTOTLT", "HRTOTLM", "HRTOTLW")\
.show()


+------+----+------------+------------+------------+
|UNITID|Year|sum(HRTOTLT)|sum(HRTOTLM)|sum(HRTOTLW)|
+------+----+------------+------------+------------+
|141264|2019|        2319|        1106|        1213|
|151467|2019|         135|           9|         126|
|155025|2019|        1341|         632|         709|
|157304|2019|         501|         169|         332|
|188207|2019|          36|           0|          36|
|197285|2019|         198|          99|          99|
|199263|2019|         189|         105|          84|
|199449|2019|         693|         287|         406|
|210146|2019|         793|         428|         365|
|213376|2019|         157|          57|         100|
|218441|2019|        1099|         623|         476|
|219347|2019|         850|         633|         217|
|221184|2019|        1071|         469|         602|
|222992|2019|        4186|        1813|        2373|
|242626|2019|         419|         161|         258|
|243443|2019|         335|         105|       

In [ ]:
df.groupBy("department","state") \
    .sum("salary","bonus") \
    .show(false)

In [8]:
# Drop duplicates
institution_size_df.distinct().show()

+----+------+-------+-------+-------+--------+
|YEAR|UNITID|HRTOTLT|HRTOTLM|HRTOTLW|INSTSIZE|
+----+------+-------+-------+-------+--------+
|2019|100663|    397|    205|    192|       5|
|2019|100751|    408|    231|    177|       5|
|2019|102049|     49|     14|     35|       3|
|2019|102313|     31|     18|     13|       2|
|2019|104179|    781|    536|    245|       5|
|2019|104717|     89|     41|     48|       5|
|2019|107600|     20|      8|     12|       1|
|2019|107974|      8|      2|      6|       2|
|2019|107983|     41|     24|     17|       2|
|2019|110404|      2|      1|      1|       2|
|2019|112251|     21|      8|     13|       2|
|2019|112260|      5|      1|      4|       2|
|2019|117724|      1|      0|      1|       4|
|2019|117946|      3|      1|      2|       3|
|2019|119137|     41|     17|     24|       4|
|2019|120865|     22|     12|     10|       1|
|2019|121707|     54|     30|     24|       2|
|2019|122296|    171|     42|    129|       2|
|2019|122454|

In [33]:
# Create the appointment_type table
appointment_type_df = highered_data_df.select(["YEAR", "UNITID", "ARANK", "HRTOTLT", "HRTOTLM", "HRTOTLW"])
appointment_type_df.show()

+----+------+-----+-------+-------+-------+
|YEAR|UNITID|ARANK|HRTOTLT|HRTOTLM|HRTOTLW|
+----+------+-----+-------+-------+-------+
|2019|100654|    0|    242|    131|    111|
|2019|100654|    0|    242|    131|    111|
|2019|100654|    1|     49|     37|     12|
|2019|100654|    2|     50|     34|     16|
|2019|100654|    3|     98|     46|     52|
|2019|100654|    4|     38|     11|     27|
|2019|100654|    5|      7|      3|      4|
|2019|100654|    0|     99|     68|     31|
|2019|100654|    1|     49|     37|     12|
|2019|100654|    2|     32|     22|     10|
|2019|100654|    3|     18|      9|      9|
|2019|100654|    0|     39|     16|     23|
|2019|100654|    2|      9|      5|      4|
|2019|100654|    3|     30|     11|     19|
|2019|100654|    0|    104|     47|     57|
|2019|100654|    2|      9|      7|      2|
|2019|100654|    3|     50|     26|     24|
|2019|100654|    4|     38|     11|     27|
|2019|100654|    5|      7|      3|      4|
|2019|100654|    0|    104|     

In [38]:
appointment_type_df.dtypes


[('YEAR', 'int'),
 ('UNITID', 'int'),
 ('ARANK', 'int'),
 ('HRTOTLT', 'int'),
 ('HRTOTLM', 'int'),
 ('HRTOTLW', 'int')]

In [51]:
from pyspark.sql.types import IntegerType,StringType
from pyspark.sql.functions import col
test_df = appointment_type_df.withColumn("UNITID",col("UNITID").cast(StringType())) \
.withColumn("ARANK",col("ARANK").cast(StringType())) \
.withColumn("HRTOTLT",col("HRTOTLT").cast(StringType()))
test_df.dtypes

[('YEAR', 'int'),
 ('UNITID', 'string'),
 ('ARANK', 'string'),
 ('HRTOTLT', 'string'),
 ('HRTOTLM', 'int'),
 ('HRTOTLW', 'int')]

In [52]:
test_df.drop_duplicates().show()

+----+------+-----+-------+-------+-------+
|YEAR|UNITID|ARANK|HRTOTLT|HRTOTLM|HRTOTLW|
+----+------+-----+-------+-------+-------+
|2019|100706|    4|      2|      0|      2|
|2019|100751|    3|    369|    207|    162|
|2019|101189|    0|     26|     12|     14|
|2019|101879|    5|     38|     14|     24|
|2019|102049|    0|     87|     40|     47|
|2019|102368|    6|      1|      1|      0|
|2019|102614|    0|     23|     13|     10|
|2019|105428|    0|    148|     62|     86|
|2019|107600|    4|     13|      4|      9|
|2019|118930|    0|     29|     13|     16|
|2019|120698|    6|     12|      4|      8|
|2019|120883|    1|     22|      8|     14|
|2019|122506|    0|      2|      1|      1|
|2019|122746|    0|    123|     46|     77|
|2019|125091|    0|      1|      0|      1|
|2019|125231|    0|    219|     65|    154|
|2019|125499|    0|    214|     87|    127|
|2019|125727|    2|      1|      1|      0|
|2019|127060|    0|    274|    114|    160|
|2019|127741|    0|    113|     